In [58]:
import os
import numpy as np
from PIL import Image
import torch
import pickle
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from DirectoryIteratorWithBoundingBoxes import DirectoryIteratorWithBoundingBoxes

In [59]:
# Current working directory
cwd = os.getcwd()
print("Current Working Directory:", cwd)

# Base path
base_path = os.path.join(cwd, "data", "data", "muscima_split")

# Correct paths for train, val, and test splits
train_xml_directory = os.path.join(base_path, "train", "annotations")
train_img_directory = os.path.join(base_path, "train", "sheet_music")
train_output_path = os.path.join(base_path, "train", "bounding_boxes.pkl")

val_xml_directory = os.path.join(base_path, "val", "annotations")
val_img_directory = os.path.join(base_path, "val", "sheet_music")
val_output_path = os.path.join(base_path, "val", "bounding_boxes.pkl")

test_xml_directory = os.path.join(base_path, "test", "annotations")
test_img_directory = os.path.join(base_path, "test", "sheet_music")
test_output_path = os.path.join(base_path, "test", "bounding_boxes.pkl")

json_path = os.path.join(cwd,'MUSCIMA_class_splits.json')

Current Working Directory: /Users/costanzasiniscalchi/Documents/MS/DLCV/Sheet-Music-Parser/ModelTrainer/datasets


In [ ]:
# Check if paths exist
for path in [train_output_path, val_output_path, test_output_path]:
    if not os.path.exists(path):
        print(f"File not found: {path}")
        raise FileNotFoundError(f"The file {path} does not exist. Check your paths.")

# Load bounding boxes
with open(train_output_path, 'rb') as f:
    train_bboxes = pickle.load(f)

with open(val_output_path, 'rb') as f:
    val_bboxes = pickle.load(f)

with open(test_output_path, 'rb') as f:
    test_bboxes = pickle.load(f)


def extract_classes_from_bboxes(bbox_dict):
    """
    Extract unique class labels from bounding box annotations.

    Args:
        bbox_dict (dict): Bounding box dictionary with filenames as keys and annotations.

    Returns:
        list: Sorted list of unique class labels.
    """
    class_set = set()
    for filename, annotations in bbox_dict.items():
        # Each annotation may have a 'class' or 'label' field
        if 'class' in annotations:
            class_set.add(annotations['class'])
        elif 'label' in annotations:
            class_set.add(annotations['label'])
    return sorted(class_set)

# Example: Load the bounding box annotations
with open("train_bounding_boxes.pkl", "rb") as f:
    train_bboxes = pickle.load(f)

# Extract the classes
classes = extract_classes_from_bboxes(train_bboxes)
print("Extracted classes:", classes)
all_classes)
    return class_list

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Classes (example)
classes = extract_classes_from_bboxes(train_bboxes)

# Initialize Datasets
train_dataset = DirectoryIteratorWithBoundingBoxes(
    directory=train_img_directory,
    bounding_boxes=train_bboxes,
    target_size=(256, 256),
    transform=transform,
    class_mode="categorical",
    classes=classes
)

val_dataset = DirectoryIteratorWithBoundingBoxes(
    directory=val_img_directory,
    bounding_boxes=val_bboxes,
    target_size=(256, 256),
    transform=transform,
    class_mode="categorical",
    classes=classes
)

test_dataset = DirectoryIteratorWithBoundingBoxes(
    directory=test_img_directory,
    bounding_boxes=test_bboxes,
    target_size=(256, 256),
    transform=transform,
    class_mode="categorical",
    classes=classes
)

# Initialize DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Test DataLoader
if __name__ == "__main__":
    for batch_x, batch_y in train_loader:
        print("Batch of images shape:", batch_x.shape)
        if isinstance(batch_y, tuple):
            labels, bboxes = batch_y
            print("Batch of labels shape:", labels.shape)
            print("Batch of bounding boxes shape:", bboxes.shape)
        else:
            print("Batch of bounding boxes shape:", batch_y.shape)
        break


KeyError: 'sheet_music'